# Explanation

## Introduction

After learning about Adversarial Search in Artificial Intelligence and Game Theory, I wanted to dive deeper into its use in modern machine learning. That's when I discovered Generative Adversarial Networks (GANs).

A GAN is a type of generative model that has two main parts: a Generative model (G) and a Discriminative model (D). These components interact with each other in a way that resembles a minimax game. The Discriminative model aims to maximize its value function, while the Generative model aims to minimize its value function.

The goal of this project is to train a generative model which can produce images.

## Notations

$z$ : noise 

$x$ : input of discriminator

$G$ : generator function, $\theta_g$ : parameters of function G

$D$ : discriminator function, $\theta_d$ : parameters of function D

$P_z$ : noise distribution

$P_{data}$ : real data distribution
 
$P_g$ : generated data using G distribution

$m$ : mini batch size

## Overview of GAN

As i mentioned earlier, the GAN consists of two parts. The generative model takes noisy variables $Z$ as input and maps them to the data space. This mapping can be represented by a differentiable function $G(z;\theta_g) = x$, which can be implemented as a multilayer perceptron with parameters 𝜃𝑔. On the other hand, the discriminative model's task is to determine whether inputs belong to the data distribution or not. It operates like a classifier, labeling inputs based on their likelihood of coming from the data distribution. The function $D(x; \theta_d)$ represents the probability of $X$ belonging to the data distribution.

Now, we can combine these two components. The generator's objective is to map random noisy inputs $Z$ to $X$ in such a way that the discriminator cannot detect that they are generated samples from $P_g$. At the same time, the discriminator aims to learn the most accurate model possible, enabling it to distinguish between real and fake data.

After training this model, we will be able to generate data (such as images) that closely resemble real data. 

## Generator Network

What the generator does is similar to finding a transformation function that minimizes an objective function. In mathematical terms, let's assume that $Z$ is drawn from a distribution called $P_z$. Our goal is to find a transformation function, denoted as $G$, that can accurately estimate the real data distribution called $P_{data}$. Essentially, the generator's objective is to make the discriminator unable to distinguish between real data from $P_{data}$ and fake data from $P_g$.

To achieve this, we can define a suitable loss function for the generator using binary cross entropy.

$loss_G = -\frac{1}{m}\sum_{z} \log(1 - D(G(z)))$ 


**Note : Generative loss doesn't have term $\log(D(x))$ because when we calculate gradient of loss with respect to $\theta_g$, $\frac{\partial D}{\partial \theta_g}$ is equal to zero.**

## Discriminator Network

The discriminator is responsible for labeling the inputs by determining if they belong to the data distribution. In simpler terms, the discriminator acts like a sigmoid function that outputs the probability of an input $x$ belonging to the real data distribution $P_{data}$. It is reasonable to use binary cross entropy as the loss function for the discriminator. Therefore one possible loss function for the discriminator might be binary cross entropy. 

$loss_D = -\frac{1}{m}\sum y\log(\hat{y}) + (1-y)\log(1-\hat{y}) = -\frac{1}{m}\sum_{x,z} \log(D(x)) + \log(1-D(G(z)))$

## Loss Function

## Theoretical Formulation

In this part, let's dive into the math behind GANs.

To begin, we introduce a new loss function that leverages the adversarial nature of the network. In previous sections, the loss functions we defined were based on practical usage. However, the authors of GANs determined a value function using a minimax game, which can be expressed as follows:

$V(D,G) = E_{x \sim P_{data}}[\log(D(x))] + E_{z \sim P_{z}}[log(1-D(G(z)))]$

So, if we consider D as a maximum node and G as a minimum node, we can view the network as an optimization problem:

$min_G max_D V(D,G) = E_{x \sim P_{data}}[\log(D(x))] + E_{z \sim P_{z}}[log(1-D(G(z)))]$

This equation effectively demonstrates the competition. It is consistent with our previous definition of loss functions. Although those loss functions were separate, decreasing the loss of G will result in an increase in the loss of D, and vice versa. Summation over all xs and zs and dividing by m can be seen as an expected value.

### Global Optimality

For this part we are going to show two things : 
    
1.What is the optimal D for a fixed G.

2.The global minimum is achieved if and only if $p_g = p_{data}$.

Lets show the first one.

If G is fixed, then we only need to maximize value function based on D.

<ul>

Value function = $E_{x \sim P_{data}}[\log(D(x))] + E_{z \sim P_{z}}[log(1-D(G(z)))]$ (1)

$E[f(X)] = \int_{x} p(x) f(x) dx$ (2)

$^{(1),(2)}=> V(G,D) = \int_{x} p_{data}(x) log(D(x)) dx + \int_{z}p_z(z) log(1-D(G(z))) dz$.

$G(z)= x$ and $P_g$ and $P_{data}$ come from same domain $=> V(G,D) = \int_{x} p_{data}(x) log(D(x)) + p_{g}(x) log(1-D(x)) dx$

Now we differentiate from above equation in order to find optimal D :

$\frac{\partial V(G,D)}{\partial D} =^{(*)} \int_{x} \frac{1}{\partial D} (p_{data}(x) log(D(x)) + p_{g}(x) log(1-D(x))) dx = \int_{x} \frac{p_{data}(x)}{D(x)} - \frac{p_{g}(x)}{1-D(x)} dx =>$

$\int_{x} \frac{p_{data}(x)}{D(x)} dx = \int_{x} \frac{p_{g}(x)}{1-D(x)} dx =^{(**)} \frac{p_{data}(x)}{D(x)} = \frac{p_{g}(x)}{1-D(x)} =>$

$D^*(x) = \frac{p_{data}(x)}{p_{data}(x) + p_{g}(x)}$
</ul>

\* : I used **differentiation under the integral sign rule**. Note that we are calculating integrals over finite intervals.

\** : $\int_{a}^{x} f(x) dx = \int_{a}^{x} g(x) dx =>^{\frac{1}{\partial x}} f(x) = g(x)$, so for any x, f is equal to g.


Now we substitute D* with D in value function equation. We will use new function $C(G)$ and with using this function we are able to minimize $max V$ based on G.

<ul>

$C(G) = max_D V(G,D) = E_{x \sim P_{data}}[\log(\frac{p_{data}(x)}{p_{data}(x) + p_{g}(x)})] + E_{x \sim P_{g}}[log(\frac{p_{g}(x)}{p_{data}(x) + p_{g}(x)})]$
    
</ul>

We have to prove : The global minimum of the $C(G)$ is achieved if and only if $p_g=p_{data}$.